# Solve the robotic manipulator problem with DAEs solvers 

## Mathematical modelling

![robot](img/robot.png)

Let us consider the robotic manipulator represented in figure above and constituted by a rigid segment moving horizontally (body 1) to which a second rigid segment is articulated. The second segment could perform a rotation motion (body 2). The motion of the system is due to the application of the force $F$ applied horizontally on the first segment and the torque of rotation $T$ applied on the second segment. The inertial basis and the different coordinates are indicated in the figure. 

This system is subjected to two type of constraints 

- Path constraints :
 $$y_1 = 0$$ 
 $$\theta_1 = 0$$
- Link constraints :
 $$x_2 - b \sin \theta_2 - x_1 - a = 0$$
 $$y_2 + b \cos \theta_2 = 0$$

Using the Lagrange multiplier method, the following equations are obtained : 

\begin{align}
        m_1 \ddot{x}_1 &= F - \lambda_3 \\
        I_2 \ddot{\theta}_2 &= -\lambda_3 b \cos \theta_2 - \lambda_4 b \sin \theta_2 + T \\ 
        m_1\ddot{y}_1 &= -m_1 g + \lambda_1 \\
        I_1 \ddot{\theta}_1 &= \lambda_2 \\
        m_2 \ddot{x}_2 &= \lambda_3 \\
        m_2 \ddot{y}_2 &= -m_2 g + \lambda_4
    \end{align}

Using the motion constraints, we can determine $\lambda_1 = m_1 \; g$ and $\lambda_2 = 0$ from equations 3 and 4.

\begin{align}
\left\{\begin{aligned}
        m_1 \ddot{x}_1 &= F - \lambda_3 \\
        I_2 \ddot{\theta}_2 &= -\lambda_3 b \cos \theta_2 - \lambda_4 b \sin \theta_2 + T \\ 
        m_2 \ddot{x}_2 &= \lambda_3 \\
        m_2 \ddot{y}_2 &= -m_2 g + \lambda_4\\
        0 &= x_2 - b \sin \theta_2 - x_1 - a \\
        0 &= y_2 + b \cos \theta_2
    \end{aligned}\right.
    \end{align}
    
This is an index 3 DAE system since 3 successive derivations of the constraint(s) must be applied to obtain an ODE. Thus, deriving twice the constraint and reduce the order of the derivative gives us the following index 1 DAE system:

\begin{align}
\left\{\begin{aligned}
        \dot{x}_1 &= u \\
        m_1 \dot{u} &= F - \lambda_3 \\
        \dot{x}_2 &= v \\
        m_2 \dot{v} &= \lambda_3 \\
        \dot{y}_2 &= w \\
        m_2 \dot{w} &= -m_2 g + \lambda_4\\
        \dot{\theta}_2 &= \alpha \\
        I_2 \dot{\alpha} &= -\lambda_3 b \cos \theta_2 - \lambda_4 b \sin \theta_2 + T \\ 
        0 &= \dot{v} - b \dot{\alpha} \cos \theta_2 + b \alpha^2 \sin \theta_2 - \dot{u} \\
        0 &= \dot{w} - b \dot{\alpha} \sin \theta_2 - b \alpha^2 \cos \theta_2
    \end{aligned}\right.
    \end{align}

We replace the values of $\dot{u}, \dot{v}, \dot{w}$ and $\dot{\alpha}$ in the last two equations to obtain constraints that depend on the variables $\lambda_3$ and $\lambda_4$. 

### ODE system

To obtain an ODE system, one normally has to derive the constraints of the index 1 DAE system. By deriving the last two equations, one obtains very complex expressions. The general dynamic of an articulated mechanical system is therefore used:

$$ M(q)\ddot{q} + f(q, \dot{q}) + g(q) = G(q)u$$

where $q = \begin{bmatrix}
    x_1 \\ \theta_2
    \end{bmatrix}$ Once the general dynamic model has been derived, the last remaining step is to write the system state model :

\begin{align*}
        \dot{q} &= v \\
        \dot{v} &= M^{-1}(q)[-f(q, v) - g(q) + G(q)u]
    \end{align*}

With our problem, we have : 

- $q = \begin{bmatrix}
    x_1 \\ \theta_2
    \end{bmatrix}$

- $M(q) = \begin{bmatrix} m_1 + m_2 & m_2 b \cos \theta_2 \\
        m_2 b \cos \theta_2 & I_2 + m_2 b^2
        \end{bmatrix}$
 
- $C(q,\dot{q}) = \begin{bmatrix} 
        0 & -m_2 b \dot{\theta}_2 \sin \theta_2 \\
        0 & 0
        \end{bmatrix}$ => $f(q,\dot{q}) = C(q,\dot{q})\dot{q}$
        
- $g(q) = \begin{bmatrix} 0 \\ b m_2 g \sin \theta_2 \end{bmatrix}$

- $G(q)u = \begin{bmatrix} F \\ T \end{bmatrix}$

## Solving those systems with DAEs solvers

In [1]:
# import solvers
from solvers.bdfSolver import BDFModified
from solvers.Final import SABMScipy
from solvers.radau import RadauModified
from solvers.rosScipyFinal import Rosenbrock

In [2]:
# import other useful modules
from time import time
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp
import numpy as np

Declare the constants of the robot manipulator problem: